In [1]:
from pathlib import Path
import os

# Establece la raíz del proyecto manualmente
project_root = Path("F:/JCMDataCenter/Cursos/Evolve Academy/Data Scientist IA/Futpeak") # sobremesa
#project_root = Path("C:/Users/juanm/Desktop/FUTPEAK/Futpeak") # portátil

# Cambia el directorio de trabajo actual a esa raíz
os.chdir(project_root)

print("📁 Directorio de trabajo actual:", Path.cwd())


📁 Directorio de trabajo actual: F:\JCMDataCenter\Cursos\Evolve Academy\Data Scientist IA\Futpeak


In [2]:
import pandas as pd
import os

# Crear carpeta si no existe
os.makedirs('data/processed', exist_ok=True)

# Cargar el archivo de metadata
metadata_df = pd.read_csv('data/processed/cleaned_metadata.csv')

# Filtrar registros donde Gender sea exactamente 'male' (ignorando mayúsculas)
male_metadata_df = metadata_df[metadata_df['Gender'].str.lower() == 'male']

# Guardar el nuevo CSV en la ruta especificada
output_path = 'data/processed/male_only_metadata.csv'
male_metadata_df.to_csv(output_path, index=False)

# Confirmar cuántos registros se guardaron
print(f"Registros con género masculino guardados: {len(male_metadata_df)}")
print(f"Archivo guardado en: {output_path}")


Registros con género masculino guardados: 3344
Archivo guardado en: data/processed/male_only_metadata.csv


In [3]:
import pandas as pd
import os

# Crear carpeta de salida si no existe
os.makedirs('data/processed', exist_ok=True)

# Cargar los archivos
metadata_df = pd.read_csv('data/processed/cleaned_metadata.csv')
matchlogs_df = pd.read_csv('data/processed/cleaned_matchlogs.csv')

# Filtrar metadata por género masculino
male_metadata_df = metadata_df[metadata_df['Gender'].str.lower() == 'male']
male_ids = set(male_metadata_df['Player_ID'])

# Filtrar matchlogs por esos jugadores
filtered_matchlogs_df = matchlogs_df[matchlogs_df['Player_ID'].isin(male_ids)]

# Eliminar columnas duplicadas del matchlogs que vendrán del metadata
filtered_matchlogs_df = filtered_matchlogs_df.drop(columns=[col for col in ['Player_name', 'Position'] if col in filtered_matchlogs_df.columns])

# Merge con metadata para agregar info del jugador
merged_df = filtered_matchlogs_df.merge(
    male_metadata_df[['Player_ID', 'Player_name', 'Full_name', 'Birth_date', 'Age', 'Position', 'Footed', 'Birth_place', 'Nationality', 'Club']],
    on='Player_ID',
    how='left'
)

# Reordenar columnas
initial_cols = ['Player_name', 'Player_ID']
merged_df = merged_df[[col for col in initial_cols if col in merged_df.columns] + [col for col in merged_df.columns if col not in initial_cols]]

# Guardar el CSV final
output_path = 'data/processed/matchlogs_male_players.csv'
merged_df.to_csv(output_path, index=False)

# Confirmación
print(f"✅ CSV guardado con {len(merged_df)} registros y {len(merged_df.columns)} columnas en: {output_path}")




✅ CSV guardado con 205748 registros y 53 columnas en: data/processed/matchlogs_male_players.csv


In [4]:
print(f"Filas antes del merge: {len(filtered_matchlogs_df)}")
print(f"Filas después del merge: {len(merged_df)}")
print("Columnas nuevas añadidas:", set(merged_df.columns) - set(filtered_matchlogs_df.columns))


Filas antes del merge: 205748
Filas después del merge: 205748
Columnas nuevas añadidas: {'Footed', 'Player_name', 'Position', 'Birth_date', 'Full_name', 'Age', 'Club', 'Birth_place', 'Nationality'}


In [5]:
# 📊 1. Jugadores con más partidos registrados

merged_df.groupby(['Player_ID', 'Player_name']).size().sort_values(ascending=False).head(10)


Player_ID  Player_name     
162efffd   Rodrigo De Paul     497
01eb744d   Angel Correa        491
7a2e46a8   Alisson             478
f7036e1c   Lautaro Martinez    457
81be82e9   Weverton            455
b3e42951   Franco Armani       451
b66315ae   Gabriel Jesus       446
81442ecb   Marcos Acuna        431
d8f11068   Nahuel Guzman       429
fba79ac8   Tiago Volpi         421
dtype: int64

In [6]:
# 📊 2. Total de minutos jugados por jugador (útil para validar volumen de datos por jugador)

merged_df['Minutes'] = pd.to_numeric(merged_df['Minutes'], errors='coerce')
merged_df.groupby(['Player_ID', 'Player_name'])['Minutes'].sum().sort_values(ascending=False).head(10)


Player_ID  Player_name      
7a2e46a8   Alisson              42977.0
81be82e9   Weverton             40924.0
b3e42951   Franco Armani        40713.0
d8f11068   Nahuel Guzman        38547.0
fba79ac8   Tiago Volpi          37858.0
8c725b43   German Pezzella      34261.0
162efffd   Rodrigo De Paul      34250.0
6afd25e6   Hugo Nervo           33018.0
a59c090b   Everson              32861.0
cf5c3ef1   Carlos Izquierdoz    32128.0
Name: Minutes, dtype: float64

In [7]:
# 📊 3. Jugadores con mayor número de temporadas únicas

merged_df.groupby(['Player_ID', 'Player_name'])['Seasons'].nunique().sort_values(ascending=False).head(10)


Player_ID  Player_name      
1edc043c   Federico Carrizo     15
b4519dd4   Nicolas Blandi       15
d4814e00   Fernando Zuqui       15
cf5c3ef1   Carlos Izquierdoz    15
d5ff6084   Marcos Guilherme     15
b14f755d   Ivan Marcone         15
7c884b79   Damian Perez         14
5887a70d   Nery Dominguez       14
5b37f2f8   Walter Kannemann     14
5a43b021   Bruno Henrique       14
Name: Seasons, dtype: int64

In [8]:
# Paso 1: obtener jugadores únicos con su país
unique_players = merged_df[['Player_ID', 'Nationality']].drop_duplicates()

# Paso 2: contar jugadores por país
top_nationalities = unique_players['Nationality'].value_counts().head(10)
print(top_nationalities)



Nationality
Brazil           499
Argentina        447
France            94
Monaco             2
United States      2
Cuba               1
Italy              1
Mexico             1
Austria            1
Belgium            1
Name: count, dtype: int64


In [9]:
merged_df['Goals'] = pd.to_numeric(merged_df['Goals'], errors='coerce')
top_goals = merged_df.groupby(['Player_ID', 'Player_name'])['Goals'].sum().sort_values(ascending=False).head(10)
print(top_goals)


Player_ID  Player_name         
f7036e1c   Lautaro Martinez        209.0
9ccf4e14   German Cano             176.0
b66315ae   Gabriel Jesus           160.0
5c4f4e43   Talisca                 155.0
1b9399bc   Bernardo Cuesta         153.0
72d0e1b6   Gabriel Barbosa         152.0
facfa8cb   Kadidiatou Diani        140.0
fc2c5692   Juan Ignacio Dinenno    120.0
8c45dfe7   Anderson Lopes          118.0
7111d552   Vinicius Junior         117.0
Name: Goals, dtype: float64


In [11]:
import pandas as pd

# Cargar los archivos
raw_df = pd.read_csv("data/raw/top_10_countries_matchlogs_filtered.csv", dtype=str, low_memory=False)
cleaned_df = pd.read_csv("data/processed/cleaned_matchlogs.csv", dtype=str)

# Buscar filas con ID de Mbappé en cualquier columna del raw
mbappe_raw_rows = raw_df[raw_df.apply(lambda row: row.astype(str).str.contains("42fd9c7f", na=False).any(), axis=1)]

# Buscar filas por ID en el cleaned
mbappe_cleaned_rows = cleaned_df[cleaned_df['Player_ID'] == '42fd9c7f']

# Convertir goles a numérico y sumar
mbappe_cleaned_rows['Goals'] = pd.to_numeric(mbappe_cleaned_rows['Goals'], errors='coerce')
total_goals = mbappe_cleaned_rows['Goals'].sum()

# Resultados
print(f"▶️ Filas en raw con ID: {len(mbappe_raw_rows)}")
print(f"✅ Filas en cleaned con ID: {len(mbappe_cleaned_rows)}")
print(f"⚽ Goles totales en cleaned: {int(total_goals)}")




KeyboardInterrupt: 